In [1]:
# 따로만든 가상환경을 aistages의 주피터에 커널로 등록
#!python -m ipykernel install --user --name Tensorflow --display-name Tensorflow

In [4]:
#!pip install -U albumentations==1.1.0 --no-binary qudida, albumentations
#!pip install opencv-python==4.5.5.64
#!pip install seaborn

## 데이터 증폭

### 1. import

In [8]:
import os
import sys
from glob import glob
import requests
import random

from sklearn.model_selection import train_test_split
from time import time

import json

import torchvision
import torchvision.transforms as transforms

from albumentations import *

#============================================================================

import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np

# Built In Imports
from datetime import datetime
import warnings
import IPython
import urllib
import zipfile
import pickle
import shutil
import string
import math
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib
import plotly
import PIL
import cv2
from pycocotools.coco import COCO

# Other Imports
from tqdm.notebook import tqdm

LABELCOLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", 10)]
LABELCOLORSWOUTNOFINDING = LABELCOLORS[:8]+LABELCOLORS[9:]

### 2. json 파일로 dataFrame 생성

In [ ]:
# train.json을 불러옵니다
json_path = ${json_dir_path}

with open(json_path, "r") as json_file:
    json_data = json.load(json_file)
    print(len(json_data["annotations"]))

In [ ]:
coco = COCO('train.json')

train_df = pd.DataFrame()
image_ids = []
class_name = []
class_id = []
x_min = []
y_min = []
x_max = []
y_max = []
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

# 이미지 인덱스에 따라서
for image_id in coco.getImgIds():
    image_info = coco.loadImgs(image_id)[0]              # 로딩
    ann_ids = coco.getAnnIds(imgIds=image_info['id'])    # 어노테이션 인덱스
    anns = coco.loadAnns(ann_ids)                        # 어노테이션 로딩
    file_name = image_info['file_name']                  # 파일 이름

    for ann in anns:
        image_ids.append(file_name)
        class_name.append(classes[ann['category_id']])
        class_id.append(ann['category_id'])
        x_min.append(float(ann['bbox'][0]))
        y_min.append(float(ann['bbox'][1]))
        x_max.append(float(ann['bbox'][0]) + float(ann['bbox'][2]))
        y_max.append(float(ann['bbox'][1]) + float(ann['bbox'][3]))

train_df['image_id'] = image_ids
train_df['class_name'] = class_name
train_df['class_id'] = class_id
train_df['x_min'] = x_min
train_df['y_min'] = y_min
train_df['x_max'] = x_max
train_df['y_max'] = y_max

## 3. 부족한 클래스 증강
- bbox 만큼 크롭
- 1024x1024로 리사이징
- Affine Transform 으로 Augmentation
- 이미지 생성
- json 파일에 정보 추가

In [ ]:
object_df = pd.DataFrame()
object_df['image_id'] = train_df['image_id']
object_df['class_id'] = train_df['class_id'].values
object_df['class_name'] = train_df['class_name'].values
object_df['x_min'] = train_df['x_min'].values
object_df['x_max'] = train_df['x_max'].values
object_df['y_min'] = train_df['y_min'].values
object_df['y_max'] = train_df['y_max'].values

avesrcimg_height = 1024
avesrcimg_width = 1024

# Initialize
object_np = object_df[["image_id", "class_name", "class_id", "x_min", "x_max", "y_min", "y_max"]].to_numpy()
object_np = np.concatenate((np.array(object_np[:, :2]), np.floor(object_np[:, 2:]).astype(np.int16)), axis=1)


import PIL.ImageOps
#https://albumentations.ai/docs/api_reference/augmentations/geometric/transforms/
trfm = Compose([
        ShiftScaleRotate(p=1.0),
        HorizontalFlip(p=0.5),
        GaussNoise(),
        ColorJitter(0.1, 0.1, 0.1, 0.1),
        #RandomBrightnessContrast(brightness_limit=(-0.3, 0.3), contrast_limit=(-0.3, 0.3), p=1.0),
    ], p=1.0)


# ==========================================================================================
data_dir = ${create_dir_path}
json_path = ${train_json_path}

with open(json_path, "r") as json_file:
    json_data = json.load(json_file)
    total_count = object_df['image_id'].nunique()
    total_anno_count = len(json_data["annotations"])
    
    
    for obj in tqdm(object_np, total=object_np.shape[0]):
        if obj[1] == "Battery" or obj[1] == "Clothing" or obj[1] == "Glass" or obj[1] == "Metal" or obj[1] == "Paper pack":
            #디버깅 코드
            img_dir = os.path.join(data_dir, obj[0])
            img_file = cv2.imread(img_dir)
            cropped_img = img_file[obj[5]:obj[6]+1, obj[3]:obj[4]+1]
            resize_img = cv2.resize(cropped_img, (1024, 1024))
            
            
            img_dir = os.path.join(data_dir, obj[0])
            img_file = Image.open(img_dir)
            cropped_img = img_file.crop((obj[3], obj[5], obj[4], obj[6]))
            resize_img = cropped_img.resize((1024, 1024))
            img_file = np.array(resize_img)

            
            if obj[1] == "Battery":
                # Ver.1 = 20 배
                # Ver.2 = 7 배
                for _ in range(7):
                    changed_img = trfm(image=img_file)['image']
                    changed_img = Image.fromarray(changed_img)
                    changed_img.save("/opt/ml/dataset_gen/train/"+str(total_count) + "." + "jpg")
                    json_data["images"].append({'width': 1024, 'height': 1024,
                                              'file_name': 'train/' + str(total_count) + '.jpg',
                                              'license': 0, 'flickr_url': None, 'coco_url': None,
                                              'date_captured': '2023-05-08 13:00:00',
                                              'id': total_count
                                               })

                    json_data["annotations"].append({'image_id': obj[0],
                                                   'category_id': obj[2],
                                                   'area': 257301.66, # 일단 임의의 수를 넣어줌
                                                   'bbox': [0, 0, 1023, 1023],
                                                   'iscrowd': 0, 'id': total_anno_count
                                                    })
                    total_count += 1
                    total_anno_count += 1


            elif obj[1] == "Clothing":
                # Ver.1 = 7 배
                # Ver.2 = 4 배
                for _ in range(4):
                    changed_img = trfm(image=img_file)['image']
                    changed_img = Image.fromarray(changed_img)
                    changed_img.save("/opt/ml/dataset_gen/train/"+str(total_count) + "." + "jpg")
                    json_data["images"].append({'width': 1024, 'height': 1024,
                                              'file_name': 'train/' + str(total_count) + '.jpg',
                                              'license': 0, 'flickr_url': None, 'coco_url': None,
                                              'date_captured': '2023-05-08 13:00:00',
                                              'id': total_count
                                               })
                    
                    json_data["annotations"].append({'image_id': obj[0],
                                                   'category_id': obj[2],
                                                   'area': 257301.66, # 일단 임의의 수를 넣어줌
                                                   'bbox': [0, 0, 1023, 1023],
                                                   'iscrowd': 0, 'id': total_anno_count
                                                    })
                    total_count += 1
                    total_anno_count += 1


            elif obj[1] == "Glass" or obj[1] == "Metal" or obj[1] == "Paper pack":
                # Ver.1 = 2 배
                # Ver.2 = 2 배
                for _ in range(2):
                    changed_img = trfm(image=img_file)['image']
                    changed_img = Image.fromarray(changed_img)
                    changed_img.save("/opt/ml/dataset_gen/train/"+str(total_count) + "." + "jpg")
                    json_data["images"].append({'width': 1024, 'height': 1024,
                                              'file_name': 'train/' + str(total_count) + '.jpg',
                                              'license': 0, 'flickr_url': None, 'coco_url': None,
                                              'date_captured': '2023-05-08 13:00:00',
                                              'id': total_count})
                    
                    json_data["annotations"].append({'image_id': obj[0],
                                                   'category_id': obj[2],
                                                   'area': 257301.66, # 일단 임의의 수를 넣어줌
                                                   'bbox': [0, 0, 1023, 1023],
                                                   'iscrowd': 0, 'id': total_anno_count
                                                    })
                    total_count += 1
                    total_anno_count += 1
            
                
    with open(json_path, 'w') as outfile:
        json.dump(json_data, outfile)